In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
# Define the dataset
data = {
    'Total_Mass': [0.84, 1.60, 2.10, 1.64, 1.64, 1.55, 1.35, 1.26, 1.30],
    'Painted_Body_Colour': [0.00, 0.00, 0.14, 0.28, 0.00, 0.00, 0.14, 0.21, 0.27],
    'Painted_Gloss_Black': [0.17, 0.17, 0.12, 0.00, 0.00, 0.25, 0.06, 0.05, 0.06],
    'Grained_Plastic': [0.00, 0.00, 0.00, 0.00, 0.28, 0.00, 0.00, 0.00, 0.00],
    'Job_1_Date': [2026, 2027, 2027, 2017, 2017, 2021, 2016, 2020, 2016],
    'Country_of_Production': ['UK', 'UK', 'UK', 'SVK', 'SVK', 'UK', 'SVK', 'UK', 'UK'],
    'Vehicle_Volume': [30000, 80000, 35000, 70000, 70000, 70000, 40000, 40000, 50000],
    'Number_of_Parts_within_Construction': [2, 2, 4, 2, 2, 2, 3, 3, 3],
    'Total_Cost': [20.15, 26.70, 38.35, 19.88, 9.12, 19.10, 17.82, 32.21, 22.81]
}

df = pd.DataFrame(data)

In [3]:
# Calculate the target variables
df['Total_Mass_per_kg'] = df['Total_Cost'] / df['Total_Mass']
df['Painted_Body_Colour_per_kg'] = df['Total_Cost'] / df['Painted_Body_Colour'].replace(0, np.nan)
df['Painted_Gloss_Black_per_kg'] = df['Total_Cost'] / df['Painted_Gloss_Black'].replace(0, np.nan)
df['Grained_Plastic_per_kg'] = df['Total_Cost'] / df['Grained_Plastic'].replace(0, np.nan)


In [4]:
# Fill NaN values with zeros
df.fillna(0, inplace=True)

# Convert categorical data
df = pd.get_dummies(df, columns=['Country_of_Production'], drop_first=True)

In [5]:
# Define the features and target variables
X = df.drop(['Total_Cost', 'Total_Mass_per_kg', 'Painted_Body_Colour_per_kg', 'Painted_Gloss_Black_per_kg', 'Grained_Plastic_per_kg'], axis=1)
y = df[['Total_Mass_per_kg', 'Painted_Body_Colour_per_kg', 'Painted_Gloss_Black_per_kg', 'Grained_Plastic_per_kg']]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
 #Standardize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [7]:
# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(y_train.shape[1], activation='linear'))

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10, validation_split=0.2)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")


Epoch 1/100
1/1 [==============================] - 1s 1s/step - loss: 1.2567 - val_loss: 0.4193
Epoch 2/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1999 - val_loss: 0.4176
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1465 - val_loss: 0.4160
Epoch 4/100
1/1 [==============================] - 0s 42ms/step - loss: 1.0996 - val_loss: 0.4141
Epoch 5/100
1/1 [==============================] - 0s 35ms/step - loss: 1.0555 - val_loss: 0.4117
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.0147 - val_loss: 0.4093
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9764 - val_loss: 0.4074
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 0.9384 - val_loss: 0.4057
Epoch 9/100
1/1 [==============================] - 0s 36ms/step - loss: 0.9020 - val_loss: 0.4040
Epoch 10/100
1/1 [==============================] - 0s 36ms/step - loss: 0.8705 - val_loss: 0.4015
Epoch 11/100
1/1 [===

In [10]:
# Predict the costs for new car models
new_car_models = {
    'Total_Mass': [1.70, 1.50],
    'Painted_Body_Colour': [0.10, 0.19],
    'Painted_Gloss_Black': [0.1, 0.0],
    'Grained_Plastic': [0.0, 0.0],
    'Job_1_Date': [2028, 2028],
    'Country_of_Production_UK': [1, 1],
    'Vehicle_Volume': [40000, 60000],
    'Number_of_Parts_within_Construction': [3, 2]
}


In [11]:
new_df = pd.DataFrame(new_car_models, columns=X.columns)
new_df = scaler_X.transform(new_df)

predictions = model.predict(new_df)
predictions = scaler_y.inverse_transform(predictions)
print(f"Predicted Costs per kg: {predictions}")

1/1 [==============================] - 0s 97ms/step
Predicted Costs per kg: [[ 18.708494  138.00943   232.72104     2.1095822]
 [ 14.031092   63.58525   199.97932     4.6391373]]


In [12]:
# Calculate the total cost for the two new cars
new_car_models['Total_Cost_Predicted'] = []

for i in range(len(new_car_models['Total_Mass'])):
    total_cost = (
        new_car_models['Total_Mass'][i] * predictions[i, 0] +
        new_car_models['Painted_Body_Colour'][i] * predictions[i, 1] +
        new_car_models['Painted_Gloss_Black'][i] * predictions[i, 2] +
        new_car_models['Grained_Plastic'][i] * predictions[i, 3]
    )
    new_car_models['Total_Cost_Predicted'].append(total_cost)

print(f"Predicted Total Costs: {new_car_models['Total_Cost_Predicted']}")

Predicted Total Costs: [68.87748699188234, 33.12783519744873]
